In [313]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [314]:
import os
import csv
import sklearn
import pandas as pd
import numpy as np
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from numpy import array
from sklearn import tree, metrics
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [315]:
# importing the datasets
players_21_dataframe = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Mid-semester Project Work/players_21.csv")
players_22_dataframe = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Mid-semester Project Work/players_22.csv")

<ipython-input-315-c4f0e47e430f>:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22_dataframe = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Mid-semester Project Work/players_22.csv")


In [316]:
players_21_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 15.9+ MB


In [317]:
# finding the null values
null_columns_21 = players_21_dataframe.columns[players_21_dataframe.isnull().any()]

In [318]:
# removing objects from the dataset
numeric_dataset_21 = players_21_dataframe.select_dtypes(exclude = ["object"])

In [319]:
# finding the mean of each column in the dataset
columns_mean_21 = numeric_dataset_21.mean()

In [320]:
# replacing the null values with the mean of their respective columns
numeric_dataset_21.fillna(columns_mean_21, inplace = True)

In [321]:
# removing numeric columns from the dataset
categorical_dataset_21 = players_21_dataframe.select_dtypes(include=["object"])

In [322]:
from pandas.core.arrays import categorical

In [323]:
# encoding each cateorical column
label_encoder = LabelEncoder()
for column in categorical_dataset_21:
    if categorical_dataset_21[column].dtype == 'object':
        categorical_dataset_21[column] = label_encoder.fit_transform(categorical_dataset_21[column])

In [324]:
# combinig both the numeric and encoded categorical datasets
combined_dataset_21 = pd.concat([numeric_dataset_21, categorical_dataset_21], axis=1)

In [325]:
# choosing the attribute to predict
target_column_21 = "overall"

In [326]:
# finding the correlation between the columns and what we want to predict
correlations_21 = combined_dataset_21.corrwith(players_21_dataframe[target_column_21])
sorted_correlations = correlations_21.sort_values(ascending = True)

In [327]:
# choosing columns with the highest correlations
desired_columns_21 = ["potential", "value_eur", "wage_eur", "age", "release_clause_eur", "shooting", "passing", "dribbling",
                      "physic", "attacking_short_passing", 'skill_long_passing', "movement_reactions", "power_shot_power",
                      "mentality_vision", "mentality_composure", "mentality_positioning"]

In [328]:
# creating a datasset with the desired columns
selected_dataset_21 = numeric_dataset_21[desired_columns_21]

In [329]:
from sklearn.preprocessing import StandardScaler

In [330]:
# scaling our dataset
SS = StandardScaler()
selected_dataset_21 = SS.fit_transform(selected_dataset_21)

In [331]:
# saving the scaled model
import pickle
pickle_out = open("scaler.pkl", "wb")
pickle.dump(SS, pickle_out)
pickle_out.close()

In [332]:
# giving a new variable to our dataset
X_data = pd.DataFrame(selected_dataset_21)

In [333]:
# giving a new variable to our target column
Y_data = numeric_dataset_21["overall"]

In [334]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_data, Y_data, test_size = 0.2, random_state = 42)

In [335]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

### **RANDOM FOREST**

In [336]:
# training our dataset with a random forest model
random_forest_model = RandomForestRegressor(n_estimators = 300, random_state = 42)
random_forest_model.fit(Xtrain, Ytrain)

RandomForestRegressor(n_estimators=300, random_state=42)

In [337]:
# testing the model
random_forest_prediction = random_forest_model.predict(Xtest)
random_forest_prediction = pd.DataFrame(random_forest_prediction)

In [338]:
# calculating the accuracy
random_forest_MAE = mean_absolute_error(Ytest, random_forest_prediction)
random_forest_MAE = mean_squared_error(Ytest, random_forest_prediction)
print(f"Mean Absolute Error: {random_forest_MAE:.4f}")
print(f"Mean Squared Error: {random_forest_MAE:.4f}")

Mean Absolute Error: 0.2192
Mean Squared Error: 0.2192


In [339]:
from sklearn.ensemble import GradientBoostingRegressor

### **GRADIENT BOOST**

In [340]:
#raining our dataset with a gradient boost
gradient_boost_model = GradientBoostingRegressor(n_estimators = 100, learning_rate = 0.1, random_state = 42)
gradient_boost_model.fit(Xtrain, Ytrain)

GradientBoostingRegressor(random_state=42)

In [341]:
# testing the model
gradient_boost_prediction = gradient_boost_model.predict(Xtest)
gradient_boost_prediction = pd.DataFrame(gradient_boost_prediction)

In [342]:
# calculating the accuracy
gradient_boost_MAE = mean_absolute_error(Ytest, gradient_boost_prediction)
gradient_boost_MAE = mean_squared_error(Ytest, gradient_boost_prediction)
print(f"Mean Absolute Error: {gradient_boost_MAE:.4f}")
print(f"Mean Squared Error: {gradient_boost_MAE:.4f}")

Mean Absolute Error: 0.6480
Mean Squared Error: 0.6480


### **XGB BOOST**

In [343]:
import xgboost as xgb
from xgboost import XGBRegressor

In [344]:
# training our dataset with an XG boost model
xgboost_model = xgb.XGBRegressor(objective = "reg:squarederror", random_state = 42)
xgboost_model.fit(Xtrain, Ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [345]:
# testing the model
xgboost_prediction = xgboost_model.predict(Xtest)
xgboost_prediction = pd.DataFrame(xgboost_prediction)

In [346]:
# calculating the accuracy
xgboost_MAE = mean_absolute_error(Ytest, xgboost_prediction)
xgboost_MSE = mean_squared_error(Ytest, xgboost_prediction)
print(f"Mean Absolute Error: {xgboost_MAE:.4f}")
print(f"Mean Squared Error: {xgboost_MSE:.4f}")

Mean Absolute Error: 0.3491
Mean Squared Error: 0.2739


In [347]:
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# **VOTING REGRESSOR**

In [348]:
# training our dataset with a voting regressor model
linear_regressor_model = LinearRegression()
svr_model = SVR(kernel = "linear")
random_forest_model = RandomForestRegressor(n_estimators = 300, random_state = 42)
gradient_boost_model = GradientBoostingRegressor(n_estimators = 100, learning_rate = 0.1, random_state = 42)
xgboost_model = xgb.XGBRegressor(objective = "reg:squarederror", random_state = 42)

In [349]:
# testing the model
voting_regressor_model = VotingRegressor(estimators = [
    ("linear regressor", linear_regressor_model),
    ("svr", svr_model),
    ("random forest", random_forest_model),
    ("gradient boost", gradient_boost_model),
    ("xgboost", xgboost_model)
])
voting_regressor_model

VotingRegressor(estimators=[('linear regressor', LinearRegression()),
                            ('svr', SVR(kernel='linear')),
                            ('random forest',
                             RandomForestRegressor(n_estimators=300,
                                                   random_state=42)),
                            ('gradient boost',
                             GradientBoostingRegressor(random_state=42)),
                            ('xgboost',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsamp...
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=42, ...))])

In [350]:
# calculating the accuracy
models = [linear_regressor_model, svr_model, random_forest_model, gradient_boost_model, xgboost_model, voting_regressor_model]
for model in models:
    model.fit(Xtrain, Ytrain)
    Y_prediction = model.predict(Xtest)
    Y_MAE = mean_absolute_error(Ytest, Y_prediction)
    Y_MSE = mean_absolute_error(Ytest, Y_prediction)
    Y_RMSE = np.sqrt(Y_MSE)
    r2 = r2_score(Ytest, Y_prediction)

    print(f"Model: {model.__class__.__name__}")
    print(f"Mean Absolute Error: {Y_MAE:.4f}")
    print(f"Mean Squared Error: {Y_MSE:.4f}")
    print(f"Root Mean Squared Error: {Y_RMSE:.4f}")
    print(f"R-squared (R^2): {r2:.4f}")
    print()

Model: LinearRegression
Mean Absolute Error: 1.5419
Mean Squared Error: 1.5419
Root Mean Squared Error: 1.2417
R-squared (R^2): 0.9162

Model: SVR
Mean Absolute Error: 1.5384
Mean Squared Error: 1.5384
Root Mean Squared Error: 1.2403
R-squared (R^2): 0.9158

Model: RandomForestRegressor
Mean Absolute Error: 0.2538
Mean Squared Error: 0.2538
Root Mean Squared Error: 0.5038
R-squared (R^2): 0.9953

Model: GradientBoostingRegressor
Mean Absolute Error: 0.5847
Mean Squared Error: 0.5847
Root Mean Squared Error: 0.7647
R-squared (R^2): 0.9863

Model: XGBRegressor
Mean Absolute Error: 0.3491
Mean Squared Error: 0.3491
Root Mean Squared Error: 0.5908
R-squared (R^2): 0.9942

Model: VotingRegressor
Mean Absolute Error: 0.7320
Mean Squared Error: 0.7320
Root Mean Squared Error: 0.8555
R-squared (R^2): 0.9806



In [351]:
from sklearn.model_selection import GridSearchCV

In [352]:
# Find the best combination of hyperparameters
random_forest = RandomForestRegressor()
param_grid = {
    "n_estimators": [100, 200, 300, 400],
    "max_depth": [None, 10, 20, 30],
}

In [353]:
# Performing hyperparameter tuning on the random regressor model
grid_search = GridSearchCV(estimator = random_forest, param_grid = param_grid, n_jobs = -1, cv = 3, scoring = "neg_mean_absolute_error")
grid_search.fit(X_data, Y_data)

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'n_estimators': [100, 200, 300, 400]},
             scoring='neg_mean_absolute_error')

In [354]:
# Selecting the best hyperparameters from the grid search
best_hyperparameters = grid_search.best_params_
better_random_forest = RandomForestRegressor(**best_hyperparameters)
better_random_forest.fit(Xtrain, Ytrain)

RandomForestRegressor(n_estimators=200)

In [355]:
Y_prediction = better_random_forest.predict(Xtest)
Y_prediction = pd.DataFrame(Y_prediction)

In [356]:
Y_MAE = mean_absolute_error(Ytest, Y_prediction)
Y_MSE = mean_squared_error(Ytest, Y_prediction)
r2 = r2_score(Ytest, Y_prediction)
rmse = np.sqrt(Y_MSE)

In [357]:
# calculating the accuracy
print(f'Mean Absolute Error: {Y_MAE:.4f}')
print(f'Mean Squared Error (MSE): {Y_MSE:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'R-squared (R^2): {r2:.4f}')

Mean Absolute Error: 0.2553
Mean Squared Error (MSE): 0.2205
Root Mean Squared Error (RMSE): 0.4695
R-squared (R^2): 0.9953


In [358]:
players_22_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 16.1+ MB


In [359]:
# finding the null values
null_columns_22 = players_22_dataframe.columns[players_22_dataframe.isnull().any()]

In [360]:
# removing objects from the dataset
numeric_dataset_22 = players_22_dataframe.select_dtypes(exclude = ["object"])

In [361]:
# finding the mean of each column in the dataset
columns_mean_22 = numeric_dataset_22.mean()

In [362]:
# replacing the null values with the mean of their respective columns
numeric_dataset_22.fillna(columns_mean_22, inplace = True)

In [363]:
# choosing the attribute to predict
target_column_22 = "overall"

In [364]:
# finding the correlation between the columns and what we want to predict
correlations_22 = numeric_dataset_22.corrwith(players_22_dataframe[target_column_22])
sorted_correlations_22 = correlations_22.sort_values(ascending = True)

In [365]:
# choosing columns with the highest correlations
desired_columns_22 = ["potential", "value_eur", "wage_eur", "age", "release_clause_eur", "shooting", "passing", "dribbling",
                      "physic", "attacking_short_passing", 'skill_long_passing', "movement_reactions", "power_shot_power",
                      "mentality_vision", "mentality_composure", "mentality_positioning"]

In [366]:
# creating a datasset with the desired columns
selected_dataset_22 = numeric_dataset_22[desired_columns_22]

In [367]:
from sklearn.preprocessing import StandardScaler

In [368]:
# scaling our dataset
SS = StandardScaler()
selected_dataset_22 = SS.fit_transform(selected_dataset_22)

In [369]:
X_data = pd.DataFrame(selected_dataset_22)

In [370]:
better_random_forest.predict(Xtest)

array([59.905, 63.99 , 57.985, ..., 72.65 , 62.975, 61.415])

In [371]:
# saving the choosen model
import pickle
pickle_out = open("better_random_forest.pkl", "wb")
pickle.dump(better_random_forest, pickle_out)
pickle_out.close()